# Membership Privacy Risk Score Examples

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/privML/privacy-evaluator/blob/team1sprint4/notebooks/membership_privacy_risk_score.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/team1sprint4/notebooks/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/privML/privacy-evaluator/blob/team1sprint4/notebooks/membership_privacy_risk_score.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/team1sprint4/notebooks/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

In this notebook, we want to show you how to use the `privacy-evaluator` tool to get Privacy Risk Scores for data points for either a PyTorch or Tensorflow model.

## Setup

First, you should set the notebook's runtime to use a GPU (e.g. if Colab is used go to ***Runtime > Change runtime type > Hardware accelerator***). Now we can install the `privacy-evaluator` package and import all needed modules.

In [ ]:
!pip3 install git+https://github.com/privML/privacy-evaluator@team1sprint4

In [ ]:
import tensorflow as tf
import torch
import numpy as np

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti
import privacy_evaluator.models.tf.dcti.dcti as tf_dcti 

from privacy_evaluator.datasets.tf.cifar10 import TFCIFAR10
from privacy_evaluator.datasets.torch.cifar10 import TorchCIFAR10

from privacy_evaluator.classifiers.classifier import Classifier

from privacy_evaluator.metrics.privacy_risk_score import * 
from privacy_evaluator.output.user_output_privacy_score import UserOutputPrivacyScore

## Compute Membership Privacy Risk Scores

Now we can start with computing Privacy Risk Scores. Therefore, we prepared two similar paths: one for the PyTorch model and one for the TensorFlow model. For both paths, we use simple neural networks trained on the CIFAR-10 dataset and implement a Lightweight Deep Convolutional Neural Network architecture (for more details, please read the following paper: https://www.scitepress.org/Papers/2018/67520/67520.pdf).

### PyTorch

We start the evaluation with the PyTorch model.

#### Load CIFAR10 Dataset

Before we can start computing the membership privacy risk scores, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the PyTorch model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TorchCIFAR10.numpy()

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `torch.nn.CrossEntropyLoss`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize PyTorch model as a Classifier
target_model = Classifier(
    torch_dcti.load_dcti(), # PyTorch DCTI
    loss=torch.nn.CrossEntropyLoss(reduction="none"), # Loss function of the PyTorch model
    nb_classes=TorchCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TorchCIFAR10.INPUT_SHAPE # Input shape of CIFAR10 dataset
)

#### Compute privacy risk score

As a next step, we want to compute the privacy risk scores. To do so, we input the target model and the data points which should be evaluated to the respective function. The given data points are separated into a train and tests set. The train set contains of the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The test set contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model. As a result, we get privacy risk scores for each data point, separated into train and test scores. The resulting values indicate the probability of a data point being a member or not.

In [ ]:
# Compute membership privacy risk score for the PyTorch model
(
    train_privacy_risk_score, 
    test_privacy_risk_score
) = compute_privacy_risk_score(
    target_model, 
    x_train[:100], 
    y_train[:100], 
    x_test[:100], 
    y_test[:100]
)

#### Get human-readable privacy risk score statistics

Besides the privacy risk scores, we can create more human-readable statistics. Therefore we generate an output by providing the privacy risk scores and the true labels of the data points for which we computed the privacy risk scores. This output can then be visualized in two separate ways.

In [ ]:
# Create user output and plot histogram for train dataset
output = UserOutputPrivacyScore(
    np.argmax(y_train[:100], axis=1),
    train_privacy_risk_score, 
)

The first way to visualise the privacy risk scores is as a histogram. The histogram shows the distribution of the k-top data points with the highest privacy risk scores per class.

In [ ]:
# Plot absolut values 
labels, count = output.histogram_top_k(range(10),50,label_names=np.array(['airplane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']))

As a second option, you can visualise the privacy risk scores again as a histogram of the distribution of the k-top data points with the highest privacy risk scores per class, but this time the values are relative to the size of respective classe

In [ ]:
# Plot relative values 
labels, count = output.histogram_top_k_relative(range(10), k=50)

### TensorFlow

Now we do the same with the TensorFlow model.

#### Load CIFAR10 Dataset

Again, before we can start computing the membership privacy risk scores, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the TensorFlow model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TFCIFAR10.numpy()

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `tf.keras.losses.CategoricalCrossentropy`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize TensorFlow target model
target_model = Classifier(
    tf_dcti.load_dcti(), # TensorFlow DCTI
    loss=tf.keras.losses.CategoricalCrossentropy(), # Loss function of the TensorFlow target model
    nb_classes=TFCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TFCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Compute privacy risk score

As a next step, we want to compute the privacy risk scores. To do so, we input the target model and the data points which should be evaluated to the respective function. The given data points are separated into a train and tests set. The train set contains of the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The test set contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model. As a result, we get privacy risk scores for each data point, separated into train and test scores. The resulting values indicate the probability of a data point being a member or not.

In [ ]:
# Compute privacy risk score for the TensorFlow target model
(
    train_privacy_risk_score, 
    test_privacy_risk_score
) = compute_privacy_risk_score(
    target_model, 
    x_train[:100], 
    y_train[:100], 
    x_test[:100], 
    y_test[:100]
)

#### Get human-readable privacy risk score statistics

Besides the privacy risk scores, we can create more human-readable statistics. Therefore we generate an output by providing the privacy risk scores and the true labels of the data points for which we computed the privacy risk scores. This output can then be visualized in two separate ways.

In [ ]:
# Create user output and plot histogram for train dataset
output = UserOutputPrivacyScore(
    np.argmax(y_train[:100], axis=1), 
    train_privacy_risk_score, 
)

The first way to visualise the privacy risk scores is as a histogram. The histogram shows the distribution of the k-top data points with the highest privacy risk scores per class.

In [ ]:
# Plot absolut values 
labels, count = output.histogram_top_k(range(10), 50)

As a second option, you can visualise the privacy risk scores again as a histogram of the distribution of the k-top data points with the highest privacy risk scores per class, but this time the values are relative to the size of respective classe

In [ ]:
# Plot relative values 
labels, count = output.histogram_top_k_relative(range(10), 50)